# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

# nltk
import nltk
nltk.download(['punkt', 'wordnet'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# sklean

In [ ]:
from sklearn.multioutput import MultiOutputClassifier

In [ ]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
df = pd.read_sql("messages_categorized", engine)
X = df.iloc[:, 0]
Y = df.iloc[:, 3:]

### 2. Write a tokenization function to process your text data

In [ ]:
def tokenize(text):
    tokens = word_tokenize(text)

    lemmatizer = WordNetLemmatizer() # Initialize lemmatizer
    tokens_cleaned = [lemmatizer.lemmatize(token).lower().strip() for token in tokens]

    return tokens_cleaned

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier


In [ ]:
pipeline = Pipeline([
    ("countVect", CountVectorizer(tokenizer=tokenize)),
    ("tfidf", TfidfTransformer()),
    ("clf", MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.3, random_state=42)

In [ ]:
#pipeline.fit(X_train, y_train)


In [ ]:
#y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
#for col_index in range(y_test.shape[1]):
#    print(Y.columns[col_index])
#    scores = classification_report(y_test.values[:, col_index], y_pred[:, col_index])
#    print(scores)
#    print("*" * 50)

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import recall_score, make_scorer

def recall_score_avg(y_test, y_pred, average='macro'):
    recall_scores = [recall_score(y_test.values[:,i], y_pred[:,i], average='macro') for i in range(y_pred.shape[1])]
    
    return sum(recall_scores) / len(recall_scores)

recall_scorer = make_scorer(recall_score_avg, greater_is_better=True)

parameters = {
        #'countVect__ngram_range': [(1, 1), (1, 2)],
        'countVect__max_df': [0.5], #, 1.0],
        #'countVect__max_features': [None, 5000, 10000],
        'tfidf__use_idf': [True], #, False],
        'clf__estimator__n_estimators': [50, 200, 500],
        #'clf__estimator__max_features': ["auto", None],
        'clf__estimator__min_samples_split': [2, 4, 8]
    }

 
# Using recall to avoid missing important and potentially survival relevant messages at the cost 
# of false positives.
cv = GridSearchCV(pipeline, param_grid=parameters, verbose=5, scoring=recall_scorer, cv=3, n_jobs=-1)
cv.fit(X_train, y_train)

print(cv.cv_results_)

In [ ]:
print(cv.best_params_)

In [ ]:
print("\nBest Parameters:", cv.best_params_)


In [ ]:
model_best = cv.best_estimator_

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

In [ ]:
class UpperCaseWordsRatioExtractor(BaseEstimator, TransformerMixin):

    def get_upper_case_words_ratio(self, text):
        tokens = tokenize(text)
        if len(tokens) < 1:
            return 0
        return sum([word.isupper() for word in tokens]) / len(tokens)

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_with_upper_case_ratio = pd.Series(X).apply(self.get_upper_case_words_ratio)
        return pd.DataFrame(X_with_upper_case_ratio)
        

In [ ]:
# Add feature: Indicator if contains all cap words

pipeline_2 = Pipeline([
    ("features", FeatureUnion([
        ("text_pipeline", Pipeline([
            ("countVect", CountVectorizer(tokenizer=tokenize)),
            ("tfidf", TfidfTransformer())])
        ),
        ("upper_case_word", UpperCaseWordsRatioExtractor())
    ])),
    ("clf", MultiOutputClassifier(RandomForestClassifier()))
    ])

In [ ]:
pipeline_2.fit(X_train, y_train)

In [ ]:
y_pred_2 = pipeline_2.predict(X_test)

In [ ]:
y_pred_train = pipeline_2.predict(X_train)

In [ ]:
for col_index in range(y_pred_2.shape[1]):
    print(Y.columns[col_index])
    scores_training = classification_report(y_train.values[:, col_index], y_pred_train[:, col_index])
    scores_1 = classification_report(y_test.values[:, col_index], y_pred[:, col_index])
    scores_2 = classification_report(y_test.values[:, col_index], y_pred_2[:, col_index])
    print("Training:")
    print(scores_training)
    print("-" * 50)
    print("Original:")
    print(scores_1)
    print("-" * 50)
    print("New:")
    print(scores_2)
    print("=" * 50)

### 9. Export your model as a pickle file

In [ ]:
pd.to_pickle(model_best, "prediction_model_v1")

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.